Automate PowerPoint

In [17]:
print(df.columns)

Index(['Segment', 'Country', 'Product', 'Discount Band', 'Units Sold',
       'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts',
       ' Sales', 'COGS', 'Profit', 'Date', 'Month Number', 'Month Name',
       'Year'],
      dtype='object')


In [22]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches

# Set the script's working directory manually
script_dir = os.getcwd()

input_folder = os.path.join(script_dir, 'input')
charts_folder = os.path.join(script_dir, 'charts')
ppt_file = 'financial_data.pptx'

# Create the charts folder if it doesn't exist
if not os.path.exists(charts_folder):
    os.mkdir(charts_folder)

# Create a new PowerPoint presentation
prs = Presentation()

# Iterate through all Excel files in the input folder
for excel_file in os.listdir(input_folder):
    if not excel_file.endswith('.xlsx'):
        continue

    # Read the financial data from the first worksheet of the Excel file
    file_path = os.path.join(input_folder, excel_file)
    df = pd.read_excel(file_path, sheet_name=0, usecols="A:P")
    df = df.dropna()

    # Strip any extra spaces in column names
    df.columns = df.columns.str.strip()

    # Ensure only numeric columns are used in the sum operation
    numeric_columns = df.select_dtypes(include=['number'])

    # Group by 'Product' in the original DataFrame and sum only numeric columns
    grouped = df.groupby('Product')[numeric_columns.columns].sum()

    # Reset the index so 'Product' becomes a column, not an index
    grouped = grouped.reset_index()

    # Create a chart using the seaborn library
    # Use the 'Product' column as x and 'Sales' column as y
    if 'Sales' in grouped.columns:
        sns.barplot(x='Product', y='Sales', data=grouped)
        plt.title(excel_file)
        plt.xlabel('Product')
        plt.ylabel('Sales')
        plt.tight_layout()

        # Save the chart to the charts folder
        chart_file = excel_file.replace('.xlsx', '.png')
        chart_path = os.path.join(charts_folder, chart_file)
        plt.savefig(chart_path)
        plt.close()  # Close the plot to avoid overlap when looping

        # Add a slide to the PowerPoint presentation and insert the chart and title
        slide = prs.slides.add_slide(prs.slide_layouts[5])
        title = slide.shapes.title
        title.text = excel_file.replace('.xlsx','')

        chart_file = chart_path
        left = Inches(0.5)
        top = Inches(1)
        width = Inches(9)
        height = Inches(6)
        slide.shapes.add_picture(chart_file, left, top, width=width, height=height)
    else:
        print(f"No 'Sales' column found in {excel_file}")

# Save the PowerPoint presentation in the same directory as the script
ppt_path = os.path.join(script_dir, ppt_file)
prs.save(ppt_path)
